In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn import mixture
%matplotlib inline

In [2]:
def outlier_detection(df, ft):
    df.boxplot(column=[ft])
    plt.grid(False)
    plt.show()

In [3]:
def outliers(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR 
    upper_bound = Q3 + 1.5 * IQR
    filt = (lower_bound>df[ft]) | (upper_bound<df[ft])
    ls = df[filt].index
    return ls

In [4]:
def remove(df, ls):
    ls = list(set(ls))
    df_clean = df.drop(index=ls)
    return df_clean 

In [117]:
def calculate_classification_data(df_clean, split_time, is_pca_on, pca_n_components, cluster_n):
    
    if is_pca_on:
        #pca transform
        pca_data = df_clean.copy()
        pca_data_standard = scale(pca_data)
        pca_model = PCA(n_components=pca_n_components, random_state=2021)
        pca_model.fit(pca_data_standard)
        for i in range(1,pca_n_components+1):
            pca_data[f'pca_{i}'] = pca_model.transform(pca_data_standard)[:,i-1]
        data_processed = pca_data.loc[:, 'pca_1':f'pca_{pca_n_components}'].copy()
    else:
        data_processed = pd.DataFrame(scale(df_clean), index=df_clean.index, columns=df_clean.columns).copy()
        
    #clustering
    df_train = data_processed[(data_processed.index>pd.to_datetime(split_time)-pd.DateOffset(years=7)) & (data_processed.index<pd.to_datetime(split_time))].copy()
#     df_train = data_processed[data_processed.index<pd.to_datetime(split_time)].copy()
    df_test = data_processed[(pd.to_datetime(split_time)<=data_processed.index) & (data_processed.index<=pd.to_datetime(split_time)+pd.DateOffset(years=1))].copy()
        
    km = KMeans(n_clusters=cluster_n, random_state=2021)
    results = km.fit_predict(df_train)
    #calculate the Silhouetter Score
    score = metrics.silhouette_score(df_train, km.labels_, metric='euclidean')
    print(f'Silhouette Score: {score}')
    df_train['label'] = results
    df_train['label'] = df_train['label'].shift(-1)
    df_train.dropna(inplace=True)
    
    df_train.reset_index(drop=True, inplace=True)
    df_test.reset_index(drop=True, inplace=True)
    
    df_train.to_csv(f'train_{split_time[0:4]}_shift_new_no_index.csv')
    df_test.to_csv(f'test_{split_time[0:4]}_shift_new_no_index.csv')
    return df_train, df_test
    
        

In [6]:
def data_cleaning(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df = df.apply(lambda x: x.pct_change())
    df.dropna(inplace=True)
    outlier_list = []
    for col in df.columns:
        outlier_list.extend(outliers(df, col))
    df_clean = remove(df, outlier_list)
    return df_clean

In [100]:
df = pd.read_csv('../Data/rawData_Feb11.csv')

In [101]:
df_base = pd.DataFrame([df['VIX'], df['Date']]).T

In [102]:
df_base['Date'] = pd.to_datetime(df_base['Date'])

In [103]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df = df[df!=0]
df = df.apply(lambda x: x.pct_change())
df.dropna(inplace=True)

In [28]:
df_clean = data_cleaning(df)

In [29]:
df_clean.reset_index(inplace=True)


In [104]:
df = df.drop('VIX',1)

In [106]:
df_use = pd.merge(df, df_base, on='Date')

In [107]:
df_use = df_use.set_index('Date')

In [108]:
df_use

,RAY,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM,VIX
Date,,,,,,,,,,,,
2011-01-31,0.007662,0.004845,0.018692,0.031901,0.019552,-0.005147,0.003038,0.013293,0.006098,0.011149,0.010589,19.53
2011-02-01,0.017057,0.018750,0.018349,-0.015403,0.019850,0.004348,0.016747,0.002449,0.009858,0.021451,0.024231,17.63
2011-02-02,-0.002660,-0.001665,0.009009,0.000992,-0.000660,-0.006047,-0.006396,0.000000,-0.001302,-0.005103,-0.007460,17.3
2011-02-03,0.002518,0.001800,0.023810,-0.003522,0.000110,0.015621,0.020545,0.002792,-0.012816,0.001775,-0.001288,16.69
2011-02-04,0.002792,0.005565,0.026163,-0.016678,0.007702,-0.002958,0.006998,-0.007308,-0.003887,-0.000591,-0.000269,15.93
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,-0.013471,-0.023181,0.019231,0.027488,0.008499,0.005864,-0.007350,-0.003984,-0.002207,-0.003748,-0.001863,31.16
2022-01-26,-0.003061,-0.001565,0.044025,0.020444,0.014607,-0.012308,-0.011760,-0.015618,-0.005398,-0.006379,-0.012650,31.96
2022-01-27,-0.007124,-0.010071,-0.030120,-0.008472,-0.020266,-0.020003,-0.032405,-0.016543,-0.008452,-0.007078,-0.011552,30.49


In [113]:
df_use[(df_use.index>pd.to_datetime('20200101')) & (df_use.index<pd.to_datetime('20210101'))]

,RAY,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM,VIX
Date,,,,,,,,,,,,
2020-01-02,0.007534,0.016697,-0.081081,0.001965,0.010011,0.003283,0.009517,-0.011747,-0.003656,0.011133,0.020281,12.47
2020-01-03,-0.006282,-0.009160,-0.176471,0.030566,-0.013451,0.015902,0.002097,0.006216,-0.000985,-0.012254,-0.018567,14.02
2020-01-06,0.003365,0.006443,-0.107143,0.003489,0.001076,0.010564,0.005111,0.001192,0.003226,0.001438,-0.002448,13.85
2020-01-07,-0.002575,-0.000139,0.160000,-0.009009,0.001254,0.003506,0.013976,-0.011150,-0.003930,-0.001795,-0.000669,13.79
2020-01-08,0.004754,0.007516,0.137931,-0.049282,0.006623,-0.008956,-0.016462,0.003065,-0.004304,0.001799,0.005805,13.45
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.002690,0.004413,-0.022989,0.002286,0.002252,0.002774,0.011264,-0.007220,0.000000,0.000506,-0.004369,21.53
2020-12-28,0.005483,0.010079,-0.023529,-0.012648,0.002809,-0.001436,0.016102,0.005961,0.002380,0.006065,0.002793,21.7
2020-12-29,-0.003945,0.000895,0.012048,0.007980,-0.004762,0.001332,-0.001261,-0.008533,0.002701,0.008372,0.014919,23.08


In [96]:
shift_tests['2019']

,RAY,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM,VIX
Date,,,,,,,,,,,,
2019-01-02,-0.049720,0.314253,0.013280,1.507308,-0.317699,0.296312,0.109216,-2.847808,-2.559878,-0.401546,0.172507,1.501074
2019-01-07,1.194403,1.363063,0.013280,0.677808,-0.403314,0.440337,-0.289987,1.110528,1.637357,0.306926,0.141396,1.118592
2019-01-08,1.513561,0.982704,3.056979,1.575405,0.630279,-0.462469,-0.035614,2.469481,-0.694016,0.889941,0.301224,0.923148
2019-01-10,0.558277,0.160137,2.637158,0.220368,-0.715404,-0.528404,-1.057247,1.584089,-0.895836,0.325290,0.659024,0.719298
2019-01-14,-1.173528,-1.395487,-2.704308,-1.370239,-0.974806,0.183005,0.250411,-0.541524,-0.010436,-0.735832,-0.987290,0.628931
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,-0.076454,-0.157444,0.013280,0.027623,0.161122,0.741842,1.239725,-0.752249,0.204094,-1.494705,0.043301,-0.728671
2019-12-24,-0.206206,-0.151195,-2.224734,0.556752,0.669793,1.589874,1.659834,0.208591,-0.010436,-0.107259,-0.337736,-0.716061
2019-12-26,0.529324,0.953260,0.013280,0.530286,0.811126,0.959283,0.459376,0.427492,0.182467,0.478389,0.663690,-0.720264


In [136]:
df_t= pd.read_csv('2021_predict_new.csv')

In [137]:
df_m = pd.read_csv('test_2021_shift_new.csv')

In [138]:
df_t['Date'] = df_m['Date']

In [139]:
df_t.to_csv('2021_predict_new_with_date.csv')

In [118]:
years = ['2018', '2019', '2020', '2021']
shift_trains = {}
shift_tests = {}

In [119]:

for year in years:
    shift_trains[year], shift_tests[year] = calculate_classification_data(df_use, f'{year}-01-01', False, 8, 2)

Silhouette Score: 0.2046899229807469
Silhouette Score: 0.19732427037096875
Silhouette Score: 0.18348805181446828
Silhouette Score: 0.2652732301522052


In [89]:
df1 = pd.read_csv('2021_shift_prediction..csv')
df2 = pd.read_csv('test_2021_boshift.csv')

In [90]:
df1['Date'] = df2['Date']

In [91]:
df1.to_csv('2021_shift_prediction_date.csv')

In [174]:
df_fractional_train_2018 = pd.read_csv('base_ffd_train_split2018-01-01.csv')
df_fractional_test_2018 = pd.read_csv('base_ffd_test_split2018-01-01_end2019-01-01.csv')
df_fractional_train_2019 = pd.read_csv('base_ffd_train_split2019-01-01.csv')
df_fractional_test_2019 = pd.read_csv('base_ffd_test_split2019-01-01_end2020-01-01.csv')
df_fractional_train_2020 = pd.read_csv('base_ffd_train_split2020-01-01.csv')
df_fractional_test_2020 = pd.read_csv('base_ffd_test_split2020-01-01_end2021-01-01.csv')
df_fractional_train_2021 = pd.read_csv('base_ffd_train_split2021-01-01.csv')
df_fractional_test_2021 = pd.read_csv('base_ffd_test_split2021-01-01_end2022-01-01.csv')

In [175]:
df_fractional_2018 = df_fractional_train_2018.append(df_fractional_test_2018)
df_fractional_2019 = df_fractional_train_2019.append(df_fractional_test_2019)
df_fractional_2020 = df_fractional_train_2020.append(df_fractional_test_2020)
df_fractional_2021 = df_fractional_train_2021.append(df_fractional_test_2021)

In [176]:
df_fractional_2018['Date'] = pd.to_datetime(df_fractional_2018['Date'])
df_fractional_2019['Date'] = pd.to_datetime(df_fractional_2019['Date'])
df_fractional_2020['Date'] = pd.to_datetime(df_fractional_2020['Date'])
df_fractional_2021['Date'] = pd.to_datetime(df_fractional_2021['Date'])

In [177]:
df_fractional_2018.set_index('Date', inplace=True)
df_fractional_2019.set_index('Date', inplace=True)
df_fractional_2020.set_index('Date', inplace=True)
df_fractional_2021.set_index('Date', inplace=True)

In [178]:
train_2018_fractional, test_2018_fractional = calculate_classification_data(df_fractional_2018, '2018-01-01', False, 8, 2)
train_2019_fractional, test_2019_fractional = calculate_classification_data(df_fractional_2019, '2019-01-01', False, 8, 2)
train_2020_fractional, test_2020_fractional = calculate_classification_data(df_fractional_2020, '2020-01-01', False, 8, 2)
train_2021_fractional, test_2021_fractional = calculate_classification_data(df_fractional_2021, '2021-01-01', False, 8, 2)

Silhouette Score: 0.26817335823466215
Silhouette Score: 0.2569776073835944
Silhouette Score: 0.26875229672351586
Silhouette Score: 0.262922477700826


In [138]:
train_2018_ret, test_2018_ret = calculate_classification_data(df, '2018-01-01', False, 8, 2)
train_2019_ret, test_2019_ret = calculate_classification_data(df, '2019-01-01', False, 8, 2)
train_2020_ret, test_2020_ret = calculate_classification_data(df, '2020-01-01', False, 8, 2)
train_2021_ret, test_2021_ret = calculate_classification_data(df, '2021-01-01', False, 8, 2)

Silhouette Score: 0.21883207190797224
Silhouette Score: 0.22574529805788388
Silhouette Score: 0.22020203602787194
Silhouette Score: 0.22411556769895105


In [172]:
train_2018_fractional, test_2018_fractional = calculate_classification_data(df_fractional_2018, '2018-01-01', False, 8, 7)
train_2019_fractional, test_2019_fractional = calculate_classification_data(df_fractional_2019, '2019-01-01', False, 8, 7)
train_2020_fractional, test_2020_fractional = calculate_classification_data(df_fractional_2020, '2020-01-01', False, 8, 7)
train_2021_fractional, test_2021_fractional = calculate_classification_data(df_fractional_2021, '2021-01-01', False, 8, 7)

Silhouette Score: 0.24689797690881368
Silhouette Score: 0.206207676469604
Silhouette Score: 0.24924434034188822
Silhouette Score: 0.2625597766963334


In [144]:
train_2018_fractional_label = pd.DataFrame(train_2018_fractional['label'], index=train_2018_fractional.index)

In [153]:
train_2019_fractional_label = pd.DataFrame(train_2019_fractional['label'], index=train_2019_fractional.index)

In [155]:
train_2020_fractional_label = pd.DataFrame(train_2020_fractional['label'], index=train_2020_fractional.index)

In [156]:
train_2021_fractional_label = pd.DataFrame(train_2021_fractional['label'], index=train_2021_fractional.index)

In [146]:
train_2018_fractional_label = train_2018_fractional_label.reset_index()

In [157]:
train_2019_fractional_label = train_2019_fractional_label.reset_index()
train_2020_fractional_label = train_2020_fractional_label.reset_index()
train_2021_fractional_label = train_2021_fractional_label.reset_index()

In [150]:
df_2018_merge = pd.merge(train_2018_ret,train_2018_fractional_label, on = 'Date')

In [158]:
df_2019_merge = pd.merge(train_2019_ret,train_2019_fractional_label, on = 'Date')
df_2020_merge = pd.merge(train_2020_ret,train_2020_fractional_label, on = 'Date')
df_2021_merge = pd.merge(train_2021_ret,train_2021_fractional_label, on = 'Date')

In [185]:
df_2020_merge

,Date,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM,label_x,label_y
0,2012-06-21,-2.120918,1.886946,-2.066774,-0.016772,-0.628154,-1.939131,-3.054138,-2.487244,-1.306626,-2.537078,-2.416435,-3.488059,0,0
1,2012-06-22,0.634273,-1.187491,0.787814,0.128276,0.305691,0.170888,0.078109,0.031336,-0.012013,0.474654,0.924793,0.221926,1,0
2,2012-06-25,-1.523985,1.426226,-1.613046,-0.103410,-0.086260,0.214710,1.323907,1.266912,-1.066989,-1.007092,-2.310861,-1.360655,0,1
3,2012-06-26,0.408345,-0.418733,0.445183,0.074065,0.042072,-0.072721,-0.845007,-0.845215,0.239709,-0.190714,0.762412,0.626104,1,0
4,2012-06-27,0.787047,-0.200082,0.429134,0.019140,0.170807,0.799328,0.201500,-0.393113,0.288970,-0.346133,0.592428,0.455954,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,2019-12-24,-0.038407,0.015725,-0.018918,-0.145633,0.156763,0.551728,1.076908,1.083011,0.183148,0.010847,0.002829,-0.171102,1,0
1890,2019-12-26,0.366951,-0.058395,0.650473,0.019140,0.150610,0.663463,0.648336,0.314847,0.320622,0.168196,0.363502,0.511335,1,0
1891,2019-12-27,-0.093925,0.681304,-0.128642,-0.068395,0.023891,-0.721542,0.275433,-0.407632,0.224643,1.390899,0.408721,0.281649,1,0
1892,2019-12-30,-0.551953,1.170700,-0.589053,0.199859,0.004960,-0.083908,0.293901,0.499708,0.052628,0.392449,-0.681013,-0.491199,0,0


In [152]:
df_2018_merge.groupby('label_y').agg(['mean', 'std']).transpose()

label_y               0         1
RAY      mean -0.023752  0.028029
         std   0.767652  0.587487
VIX      mean  0.006453 -0.025221
         std   0.991688  0.815369
QQQ      mean -0.023851  0.021848
         std   0.806848  0.621156
T10Y3M   mean  0.017075  0.020620
         std   0.075851  0.063503
CL1      mean  0.001888  0.025695
         std   0.403625  0.174893
HG1      mean -0.040217  0.031407
         std   0.985385  0.794835
GC1      mean -0.040876 -0.015069
         std   0.992271  0.979779
XAG      mean -0.037367 -0.007878
         std   0.883066  0.841307
VNQ      mean -0.010072 -0.004698
         std   0.782097  0.650068
EURtoUSD mean -0.029869  0.035802
         std   1.190951  0.847859
VXUS     mean -0.016692  0.027684
         std   0.894071  0.638045
EEM      mean -0.012538  0.013754
         std   0.910369  0.743808
label_x  mean  0.582840  0.647059
         std   0.493455  0.478299

In [159]:
df_2019_merge.groupby('label_y').agg(['mean', 'std']).transpose()

label_y               0         1
RAY      mean  0.040835 -0.083326
         std   0.620521  0.881637
VIX      mean -0.036136  0.057174
         std   0.964971  1.061627
QQQ      mean  0.049474 -0.091565
         std   0.699201  0.948831
T10Y3M   mean  0.019807  0.012382
         std   0.074939  0.100538
CL1      mean  0.027083 -0.011273
         std   0.199935  0.421733
HG1      mean  0.025125 -0.074669
         std   0.820603  1.014583
GC1      mean -0.028976 -0.027415
         std   0.872075  1.024502
XAG      mean -0.019598 -0.032086
         std   0.764244  0.900309
VNQ      mean -0.008457 -0.026471
         std   0.654996  0.843739
EURtoUSD mean  0.027888 -0.052306
         std   0.874643  1.182320
VXUS     mean  0.039594 -0.074637
         std   0.646061  0.954202
EEM      mean  0.030265 -0.060507
         std   0.762671  0.983451
label_x  mean  0.666271  0.549844
         std   0.471825  0.497897

In [160]:
df_2020_merge.groupby('label_y').agg(['mean', 'std']).transpose()

label_y               0         1
RAY      mean -0.017620  0.019255
         std   0.785650  0.679707
VIX      mean -0.001525 -0.007915
         std   0.974246  0.976870
QQQ      mean -0.017663  0.010200
         std   0.872545  0.736034
T10Y3M   mean -0.008117  0.021432
         std   1.439613  0.074240
CL1      mean  0.005930  0.028166
         std   0.374558  0.195245
HG1      mean -0.041657  0.035583
         std   0.937468  0.819714
GC1      mean -0.018721 -0.008177
         std   0.886847  0.934664
XAG      mean -0.020160 -0.001553
         std   0.801676  0.810314
VNQ      mean  0.003031 -0.018315
         std   0.738898  0.666184
EURtoUSD mean -0.025863  0.035624
         std   1.041531  0.879823
VXUS     mean -0.010903  0.023199
         std   0.830281  0.716780
EEM      mean -0.008757  0.013501
         std   0.879182  0.784360
label_x  mean  0.616451  0.647783
         std   0.486475  0.477955

In [161]:
df_2021_merge.groupby('label_y').agg(['mean', 'std']).transpose()

label_y               0         1
RAY      mean  0.026835 -0.052754
         std   0.997320  0.921294
VIX      mean -0.020805  0.047933
         std   0.956171  1.073755
QQQ      mean  0.035452 -0.066201
         std   1.018893  0.904500
T10Y3M   mean -0.018189  0.023014
         std   1.357063  0.080509
CL1      mean -0.017253  0.011578
         std   1.338321  0.204750
HG1      mean  0.006350 -0.013343
         std   0.942329  0.832168
GC1      mean  0.020526 -0.058757
         std   0.909071  1.082320
XAG      mean  0.026598 -0.057334
         std   0.901280  0.959896
VNQ      mean  0.014589 -0.061138
         std   0.993559  0.925867
EURtoUSD mean  0.007894  0.013927
         std   1.007848  0.875762
VXUS     mean  0.027297 -0.040412
         std   0.936265  0.947055
EEM      mean  0.032216 -0.053455
         std   0.951849  0.948673
label_x  mean  0.646667  0.630603
         std   0.478165  0.483015

In [45]:
for i in range(2,10):
    train_2018_fractional, test_2018_fractional = calculate_classification_data(df_fractional_2018, '2018-01-01', False, 8, i)

Silhouette Score: 0.260833399192008
Silhouette Score: 0.21574167212638495
Silhouette Score: 0.24810438308411334
Silhouette Score: 0.2605014623928107
Silhouette Score: 0.2676076560991107
Silhouette Score: 0.2724635776974476
Silhouette Score: 0.26235209510336904
Silhouette Score: 0.25985887483189585


In [47]:
train_2018_fractional, test_2018_fractional = calculate_classification_data(df_fractional_2018, '2018-01-01', False, 8, 7)

Silhouette Score: 0.2724635776974476


In [46]:
for i in range(2,10):
    train_2019_fractional, test_2019_fractional = calculate_classification_data(df_fractional_2019, '2019-01-01', False, 8, i)

Silhouette Score: 0.22169401833745367
Silhouette Score: 0.23891646438568226
Silhouette Score: 0.23678045131088796
Silhouette Score: 0.2615238360333878
Silhouette Score: 0.2686383888147033
Silhouette Score: 0.25164006192972904
Silhouette Score: 0.2546832590557218
Silhouette Score: 0.25131927130937654


In [38]:
train_2019_fractional.groupby('label').agg(['mean', 'std']).transpose()

label                  0          1
RAY      mean  64.629593  90.707386
         std   13.858881  12.313159
QQQ      mean   5.618978   9.136491
         std    1.573363   2.106877
T10Y3M   mean   0.845740   0.585299
         std    0.535854   0.392764
CL1      mean  19.905018  18.853104
         std    9.970432   6.202539
HG1      mean  48.765633  55.143979
         std   11.489690  10.098894
GC1      mean  58.082138  69.791483
         std   19.027933  13.814337
XAG      mean   2.750695   3.055290
         std    0.699488   0.601530
VNQ      mean  21.782208  23.539544
         std    2.383562   2.056173
EURtoUSD mean   0.604290   0.589465
         std    0.064339   0.046389
VXUS     mean  18.266239  20.177638
         std    1.830233   1.857043
EEM      mean  32.236383  35.059532
         std    3.488709   3.663463
VIX      mean  16.271697  13.055300
         std    4.272879   2.896452

In [39]:
train_2020_fractional.groupby('label').agg(['mean', 'std']).transpose()

label                  0          1
RAY      mean  52.030582  39.864385
         std   14.503281  14.347038
QQQ      mean   5.298176   3.651397
         std    2.160772   1.717545
T10Y3M   mean   0.374624   0.625095
         std    0.443603   0.511492
CL1      mean  20.879072  20.485502
         std    7.607518   9.309404
HG1      mean  47.807268  42.530123
         std    9.632620   9.692765
GC1      mean  74.656844  49.142573
         std   12.373442  14.254896
XAG      mean   3.287617   2.552930
         std    0.690195   0.527894
VNQ      mean  20.890065  19.329231
         std    2.474680   2.127255
EURtoUSD mean   0.593774   0.592107
         std    0.051531   0.059836
VXUS     mean  37.284119  35.778106
         std    2.982001   2.645227
EEM      mean  34.725604  32.745331
         std    3.553903   3.410512
VIX      mean  14.256151  15.582441
         std    3.386916   4.220688

In [41]:
train_2021_fractional.groupby('label').agg(['mean', 'std']).transpose()

label                  0          1
RAY      mean  47.967893  83.072648
         std   20.180686  17.179585
QQQ      mean   2.973392   7.535354
         std    2.073984   2.950188
T10Y3M   mean   0.566772   0.166370
         std    0.474510   0.263515
CL1      mean  24.597535  21.091556
         std    9.892230   7.665056
HG1      mean  37.957238  44.212668
         std    9.333156  10.217345
GC1      mean  55.236286  84.430856
         std   15.928576  22.137925
XAG      mean   2.824320   3.864922
         std    0.604250   1.328411
VNQ      mean  17.553824  18.891036
         std    2.397153   2.774818
EURtoUSD mean   0.624871   0.620154
         std    0.053253   0.032174
VXUS     mean  36.283366  38.424701
         std    3.177856   3.246084
EEM      mean  33.189243  36.276087
         std    3.820922   3.484146
VIX      mean  16.053167  19.932713
         std    7.321166   8.345014

In [18]:
train_2018_no_pca_no_shift, test_2018_no_pca_no_shift = calculate_classification_data(df, '2018-01-01', False, 8, 2)
train_2019_no_pca_no_shift, test_2019_no_pca_no_shift = calculate_classification_data(df, '2019-01-01', False, 8, 2)
train_2020_no_pca_no_shift, test_2020_no_pca_no_shift = calculate_classification_data(df, '2020-01-01', False, 8, 2)
train_2021_no_pca_no_shift, test_2021_no_pca_no_shift = calculate_classification_data(df, '2021-01-01', False, 8, 2)

Silhouette Score: 0.19091367279812835
Silhouette Score: 0.18940361917035275
Silhouette Score: 0.17958249395692993
Silhouette Score: 0.17761300426185292


In [32]:
train_2018_no_pca_no_shift, test_2018_no_pca_no_shift = calculate_classification_data(df, '2018-01-01', False, 8, 2)

Silhouette Score: 0.19091367279812835


In [60]:
train_2018['y'] = train_2018['label'].shift(1)
train_2018.dropna(inplace=True)

In [12]:
train_2018_no_pca_no_shift.reset_index(drop=True)

,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM,label
0,1.040954,-0.354611,0.424050,0.724424,1.947947,1.772630,-0.756082,0.223098,1.546021,1.438929,1.439790,1.026191,1
1,2.587361,-1.674205,2.273316,0.711376,-1.022856,1.800195,0.606812,1.410556,0.185336,2.332677,2.892800,2.476652,1
2,-0.658071,-0.230967,-0.441719,0.356039,0.006766,-0.097411,-0.885268,-0.594027,-0.121931,-0.319801,-0.852462,-0.892856,0
3,0.239252,-0.723597,0.519774,1.008676,-1.102913,0.676213,-0.441930,0.566159,-1.038919,-0.934373,-0.216051,-0.128284,1
4,0.885408,0.516463,0.417167,-0.094500,-1.148895,-0.127283,-0.091878,0.721881,1.109721,0.024565,0.256351,0.043212,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,0.154666,-0.076114,-0.203377,0.698798,0.236401,0.634621,0.118686,-0.354115,-2.514806,0.049629,-0.781074,0.596605,0
1290,-0.301856,0.647526,-0.372203,0.686665,0.062870,0.542900,0.900068,1.012473,0.742665,-0.250637,0.293151,0.799610,1
1291,-0.100126,0.507137,-0.203268,-0.747645,-0.401093,-0.007950,0.306806,0.689975,0.290011,0.590876,0.116903,0.174921,1
1292,0.121063,-0.395925,-0.058845,2.342641,0.155102,1.426621,1.120235,0.848896,0.547185,1.089187,0.116462,0.630748,1


In [ ]:
train_2018_no_pca_no_shift.groupby('label').agg(['mean', 'std']).transpose()

In [28]:
train_2019_no_pca_no_shift.groupby('label').agg(['mean', 'std']).transpose()

label                 0         1
RAY      mean -0.710110  0.616550
         std   0.666278  0.708270
VIX      mean  0.646385 -0.571512
         std   0.847098  0.760891
QQQ      mean -0.625184  0.549200
         std   0.757752  0.769057
T10Y3M   mean -0.200498  0.213523
         std   0.911040  0.889559
CL1      mean -0.352005  0.263496
         std   0.958465  0.910573
HG1      mean -0.351713  0.264235
         std   0.878178  0.956869
GC1      mean -0.167562  0.067073
         std   0.997511  0.991580
XAG      mean -0.305520  0.176099
         std   0.924383  0.933576
VNQ      mean -0.402442  0.318354
         std   0.893149  0.883418
EURtoUSD mean -0.228826  0.205477
         std   0.971937  1.032141
VXUS     mean -0.763225  0.684090
         std   0.680214  0.666837
EEM      mean -0.711112  0.661196
         std   0.745753  0.716695

In [37]:
train_2019.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.101101  0.004082
      std   2.132772  2.045445
pca_2 mean -0.026515 -0.035622
      std   1.411292  1.437818
pca_3 mean  0.006772  0.004501
      std   1.017987  1.074369
pca_4 mean  0.023457  0.045789
      std   0.941696  0.951505
pca_5 mean -0.022429  0.027400
      std   0.888139  0.832009
pca_6 mean  0.009709  0.021880
      std   0.817773  0.804553
pca_7 mean  0.001243  0.014802
      std   0.727812  0.758405
pca_8 mean -0.042481  0.011679
      std   0.694154  0.658690

In [38]:
train_2020.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.089637  0.002619
      std   2.117335  2.030744
pca_2 mean -0.020301 -0.037553
      std   1.404899  1.426434
pca_3 mean  0.003569  0.020551
      std   1.039725  1.088490
pca_4 mean  0.005504  0.040554
      std   0.936952  0.949386
pca_5 mean -0.019440  0.025084
      std   0.883818  0.833288
pca_6 mean  0.007907  0.014933
      std   0.821588  0.814081
pca_7 mean -0.001835  0.014721
      std   0.742515  0.768959
pca_8 mean -0.039503  0.007276
      std   0.693687  0.655838

In [39]:
train_2021.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.068618 -0.048819
      std   2.143774  2.027236
pca_2 mean  0.026032 -0.049989
      std   1.397406  1.426464
pca_3 mean -0.000414  0.011116
      std   1.087306  1.095950
pca_4 mean -0.009574  0.044486
      std   0.939768  0.942816
pca_5 mean -0.015625  0.013196
      std   0.893176  0.857122
pca_6 mean -0.005993  0.013042
      std   0.838490  0.815781
pca_7 mean  0.019051  0.012016
      std   0.798136  0.782206
pca_8 mean -0.025330  0.009359
      std   0.715721  0.663529

#### cluster the whole dataset

In [23]:
df_clean

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
Date,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,-0.001461,0.034115,-0.003645,-0.001406,0.000658,-0.007399,-0.006024,-0.008492,-0.013152,0.000779,0.004489,-0.005764,-0.002993
2022-01-11,0.009908,-0.051031,0.011454,0.019105,0.015022,0.014274,-0.006061,0.038221,0.017693,0.010952,0.013957,0.001540,0.003620
2022-01-14,0.000467,-0.055145,0.000407,-0.014383,0.006223,-0.000935,0.044304,0.020701,-0.027607,-0.002690,-0.005457,-0.009052,-0.003841


In [29]:
km = KMeans(n_clusters=3, random_state=2021)
results_whole = km.fit_predict(df_clean)
#calculate the Silhouetter Score
score = metrics.silhouette_score(df_clean, km.labels_, metric='euclidean')
print(f'Silhouette Score: {score}')

Silhouette Score: 0.2496869919356694


In [30]:
df_test_whole = df_clean.copy()
df_test_whole['label'] = results_whole-1

In [31]:
df_test_whole.to_csv('benchmark_data_3_cluster.csv')

In [32]:
df_test_whole.head()

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,label
Date,,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416,-1
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871,0
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888,0
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316,1
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160,0


#### find expected return and vol for each group

In [33]:

df_test_whole.groupby('label').agg(['mean', 'std']).transpose()

label                -1         0         1
RAY      mean -0.004872  0.001254  0.007005
         std   0.004299  0.003702  0.004813
VIX      mean  0.068080 -0.005807 -0.074200
         std   0.029603  0.019636  0.028418
IWV      mean -0.004897  0.001255  0.006989
         std   0.004318  0.003732  0.004838
EEM      mean -0.002378  0.001042  0.003122
         std   0.009774  0.009660  0.009642
QQQ      mean -0.005135  0.001982  0.007815
         std   0.006355  0.005831  0.006958
VXUS     mean -0.004633  0.000944  0.006007
         std   0.005290  0.005255  0.005615
T10Y3M   mean -0.007835 -0.002782  0.010282
         std   0.034408  0.031933  0.032450
CL1      mean -0.004897  0.001225  0.008320
         std   0.017619  0.017053  0.019203
HG1      mean -0.001845  0.000894  0.003025
         std   0.010279  0.010700  0.011709
GC1      mean  0.000615  0.000410 -0.000072
         std   0.006620  0.006271  0.006245
XAG      mean -0.001106  0.000764  0.001798
         std   0.011231  0.010267  0.010898
VNQ      mean -0.002816  0.000587  0.004860
         std   0.008259  0.007522  0.008222
EURtoUSD mean -0.000048  0.000113  0.000053
         std   0.003795  0.004146  0.004302

In [ ]:
#based on the above results, 1: bull market, -1: bear market 0: stable market

In [39]:
#Hierarchical Clustering
hr_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hr_cluster.fit_predict(df_clean)
hr_score = metrics.silhouette_score(df_clean, hr_cluster.labels_, metric='euclidean')
print(f'Silhouette Score: {hr_score}')

Silhouette Score: 0.22456252552673217


In [66]:
pd.to_datetime('2019-01-01')-pd.DateOffset(years=7)

Timestamp('2012-01-01 00:00:00')

In [20]:
df_predict = pd.read_csv('2018prediction_no_pca_no_shift.csv')

In [22]:
df_predict = df_predict.drop()

,label_1_PREDICTION,label_0_PREDICTION,label_PREDICTION,THRESHOLD,POSITIVE_CLASS,DEPLOYMENT_APPROVAL_STATUS,Date,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM
0,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-01-02,1.142000,-2.000259,2.110663,0.389975,-0.107476,-0.667111,0.728120,1.251256,-0.590684,1.058672,1.282395,1.908592
1,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-01-03,0.741047,-1.052835,1.071970,0.386282,1.255274,-0.614989,0.244393,-0.312143,-0.486197,-0.877660,0.850476,0.919048
2,1.000000,6.000000e-10,1,0.5,1,APPROVED,2018-01-04,0.377592,0.253400,0.012428,0.752042,0.331728,0.119851,0.320121,0.427529,-2.283447,1.038000,1.038949,0.426772
3,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-01-05,0.837449,0.112870,1.115370,1.100315,-0.632793,-0.986268,0.058669,-0.071715,-0.059976,-0.778539,0.811417,0.817093
4,0.000352,9.996484e-01,0,0.5,1,APPROVED,2018-01-08,0.081992,0.710568,0.297180,-1.395840,0.240929,-0.193938,-0.223605,-0.483644,0.528276,-1.235481,-0.204870,-0.099708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.000000,1.000000e+00,0,0.5,1,APPROVED,2018-12-10,-0.065662,-0.353676,1.172441,-0.832192,-1.977434,-1.344945,-0.374244,-0.602696,-0.931740,-0.490848,-1.350592,-1.219502
180,1.000000,0.000000e+00,1,0.5,1,APPROVED,2018-12-12,0.756320,-0.140382,0.950246,0.013280,-0.663471,0.047259,0.271590,0.976839,-2.058693,1.081663,2.082874,1.638536
181,0.230823,7.691773e-01,0,0.5,1,APPROVED,2018-12-13,-0.565669,-0.580471,-0.179974,0.013280,1.700277,-0.236886,-0.213447,0.039307,-0.886924,-0.177683,-0.333639,-0.126020
182,0.000000,1.000000e+00,0,0.5,1,APPROVED,2018-12-20,-2.861108,2.123578,-2.148039,3.098110,-1.811857,-0.699311,1.300972,0.961556,-2.005932,1.452072,-0.310920,0.729030


In [ ]:
df_predict.groupby('label_PREDICTION').agg(['mean', 'std']).transpose()